slack - get more data

only train model on HSCs

look at and compare model weights

In [1]:
import pandas as pd
import scanpy as sc
from scipy.io import mmread
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import ElasticNetCV, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

# gene-GEP mapping model fits

In [2]:
gene_GEP_mapping = pd.read_csv(r'./data/Factorized matrices from human lifetime scRNA(Gene Z-score x GEP).csv').set_index('Gene')
gene_GEP_mapping.head()

,GEP 1 (Mono/DC),GEP 2 (single donor-specific),GEP 3,GEP 4 (Elderly-HSC),GEP 5,GEP 6,GEP 7,GEP 8,GEP 9,GEP 10,GEP 11,GEP 12,GEP 13,GEP 14 (Meg),GEP 15 (DNA Replication),GEP 16,GEP 17,GEP 18,GEP 19,GEP 20,GEP 21 (Mitochondiral genes),GEP 22 (Cell cycle),GEP 23,GEP 24,GEP 25 (Baso/Mast),GEP 26 (Lymphoid),GEP 27 (Ribosomal biogenesis),GEP 28 (Fetal-HSC),GEP 29,GEP 30 (Granulocyte),GEP 31,GEP 32 (Erythroid),GEP 33 (Pan-cellular expression),GEP 34,GEP 35
Gene,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
X5S-rRNA,0.000052,0.000116,0.000263,-0.000159,0.000005,-0.000078,-0.000068,0.000133,0.000013,0.000045,-0.000144,-0.000159,0.000101,0.000155,-0.000043,0.000404,0.000694,-0.000127,-0.000320,0.000706,-0.000357,-1.419610e-04,-0.000007,-0.000191,0.000056,0.000002,0.000134,-0.000127,0.000497,0.000069,0.000093,0.000020,0.000139,-0.000119,-0.000163
X5-8S-rRNA,-0.000103,0.000006,-0.000108,-0.000053,-0.000018,-0.000056,-0.000023,-0.000036,0.000012,-0.000088,0.000249,-0.000041,-0.000026,-0.000056,-0.000039,0.000015,-0.000085,0.000005,-0.000121,-0.000155,-0.000155,-5.680000e-07,0.000009,0.000040,0.000063,-0.000019,-0.000010,-0.000085,0.000517,0.000008,-0.000006,-0.000026,0.000095,-0.000065,-0.000026
X7SK,-0.000016,0.000140,0.000461,-0.000180,-0.000021,-0.000083,-0.000011,0.000109,0.000092,0.000473,-0.000136,0.000092,0.000043,0.000287,-0.000040,0.000058,-0.000144,0.000510,-0.000146,0.000467,-0.000417,-1.021400e-04,0.000051,-0.000145,0.000062,0.000006,-0.000086,0.000201,0.000052,0.000089,0.000155,-0.000051,0.000107,-0.000216,-0.000171
A1BG,0.000171,0.000117,-0.000189,-0.000157,0.000059,0.000545,-0.000263,-0.000040,0.000030,-0.000020,-0.000174,0.000339,-0.000053,0.000133,-0.000166,-0.000044,0.000426,-0.000105,-0.000281,0.000505,-0.000388,-2.606410e-04,-0.000047,0.000385,-0.000028,0.000142,0.000048,0.000055,-0.000144,-0.000026,0.000113,-0.000094,0.000122,0.000032,-0.000051
A1BG.AS1,0.000117,0.000149,0.000394,0.000058,-0.000030,-0.000047,0.000091,0.000076,-0.000044,0.000401,-0.000060,-0.000090,-0.000006,-0.000123,-0.000162,0.000425,-0.000090,0.000057,0.000134,-0.000032,-0.000445,-1.729850e-04,-0.000015,-0.000243,0.000162,0.000269,-0.000310,0.000103,-0.000104,0.000082,0.000036,-0.000071,0.000161,-0.000280,-0.000068


In [3]:
def get_X_gene():
    return gene_GEP_mapping.drop('GEP 15 (DNA Replication)', axis=1)

def get_y_gene():
    return gene_GEP_mapping['GEP 15 (DNA Replication)']

In [4]:
X_gene_train, X_gene_test, y_gene_train, y_gene_test = train_test_split(get_X_gene(), get_y_gene(), test_size=0.2)

# alphas = np.logspace(-6, 6, 500)
# l1_ratios = [0.0, 0.01, 0.025, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.97, 1.0]

# gene_elastic = ElasticNetCV(alphas=alphas, l1_ratio=l1_ratios, cv=10, n_jobs=-1, verbose=0)
# gene_elastic.fit(X_gene_train, y_gene_train)

In [5]:
# print(f'optimal gene alpha: {gene_elastic.alpha_}')
# print(f'optimal gene l1_ratio: {gene_elastic.l1_ratio_}')
# print(f'elastic net gene test R^2: {gene_elastic.score(X_gene_test, y_gene_test)}')

In [6]:
gene_LR = LinearRegression()

gene_LR.fit(X_gene_train, y_gene_train)

LinearRegression()

In [7]:
print(f'base linear regression gene test R^2: {gene_LR.score(X_gene_test, y_gene_test)}')

base linear regression gene test R^2: 0.957096100756764


In [8]:
print(f'MSE for gene model: {mean_squared_error(y_gene_test, gene_LR.predict(X_gene_test))}')

MSE for gene model: 2.4999613298392805e-09


In [9]:
gene_weights = pd.DataFrame({
    'Feature': X_gene_train.columns,
    'Weight': gene_LR.coef_
})
gene_weights = pd.concat(
    [gene_weights, pd.DataFrame({'Feature': ['Intercept'], 'Weight': [gene_LR.intercept_]})]
)

gene_weights

,Feature,Weight
0,GEP 1 (Mono/DC),-0.233016
1,GEP 2 (single donor-specific),-0.185485
2,GEP 3,-0.029408
3,GEP 4 (Elderly-HSC),-0.640429
4,GEP 5,-0.013240
5,GEP 6,-0.011671
6,GEP 7,-0.024677
7,GEP 8,-0.037174
8,GEP 9,-0.099093
9,GEP 10,-0.024306


In [10]:
gene_weights.sort_values('Weight')

,Feature,Weight
31,GEP 33 (Pan-cellular expression),-1.300081
3,GEP 4 (Elderly-HSC),-0.640429
24,GEP 26 (Lymphoid),-0.500401
25,GEP 27 (Ribosomal biogenesis),-0.405514
19,GEP 21 (Mitochondiral genes),-0.250111
28,GEP 30 (Granulocyte),-0.233099
0,GEP 1 (Mono/DC),-0.233016
20,GEP 22 (Cell cycle),-0.229858
30,GEP 32 (Erythroid),-0.212208
26,GEP 28 (Fetal-HSC),-0.198372


# cell-GEP mapping model fits

fit model only on HSCs

In [11]:
cell_GEP_mapping = pd.read_csv(r'./data/Factorized matrices from human lifetime scRNA(GEP usage per cell).csv').set_index('Cell')
cell_GEP_mapping.head()

,GEP 1 (Mono/DC),GEP 2 (single donor-specific),GEP 3,GEP 4 (Elderly-HSC),GEP 5,GEP 6,GEP 7,GEP 8,GEP 9,GEP 10,GEP 11,GEP 12,GEP 13,GEP 14 (Meg),GEP 15 (DNA Replication),GEP 16,GEP 17,GEP 18,GEP 19,GEP 20,GEP 21 (Mitochondiral genes),GEP 22 (Cell cycle),GEP 23,GEP 24,GEP 25 (Baso/Mast),GEP 26 (Lymphoid),GEP 27 (Ribosomal biogenesis),GEP 28 (Fetal-HSC),GEP 29,GEP 30 (Granulocyte),GEP 31,GEP 32 (Erythroid),GEP 33 (Pan-cellular expression),GEP 34,GEP 35
Cell,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BM1_bcBSQM,0.036803,0.026881,0.012481,0.464846,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000645,0.002771,0.000000,0.000000,0.111207,0.0,0.001688,0.000000,0.0,0.000000,0.038902,0.000383,0.0000,0.001779,0.00000,0.065462,0.179906,0.0,0.000000,0.014643,0.010171,0.000000,0.031434,0.00000,0.000000
BM1_bcCHLB,0.010666,0.000000,0.000000,0.435595,0.000000,0.0,0.020563,0.011753,0.016536,0.017280,0.000000,0.033893,0.000681,0.029359,0.070330,0.0,0.000000,0.000000,0.0,0.000000,0.041981,0.000000,0.0065,0.003607,0.00000,0.000000,0.043055,0.0,0.000000,0.002227,0.016027,0.018264,0.221683,0.00000,0.000000
BM1_bcFZFC,0.062572,0.000000,0.006028,0.396839,0.000000,0.0,0.000000,0.007984,0.047616,0.005497,0.000000,0.000000,0.000000,0.026648,0.161894,0.0,0.000000,0.001741,0.0,0.018964,0.030627,0.003311,0.0000,0.000068,0.00000,0.002143,0.058599,0.0,0.000046,0.010492,0.000000,0.000000,0.149481,0.00945,0.000000
BM1_bcGZEW,0.045451,0.031609,0.000000,0.517612,0.000000,0.0,0.000000,0.009147,0.031807,0.000000,0.006016,0.000000,0.000000,0.016127,0.083941,0.0,0.000000,0.000000,0.0,0.000000,0.036858,0.000000,0.0000,0.000000,0.00003,0.004162,0.113646,0.0,0.000000,0.000000,0.000000,0.043661,0.000000,0.00000,0.059932
BM1_bcFMRM,0.041508,0.000000,0.000000,0.302918,0.000057,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.003737,0.000000,0.000000,0.225007,0.0,0.000000,0.010482,0.0,0.003365,0.064948,0.004206,0.0000,0.000000,0.00000,0.000000,0.102505,0.0,0.000000,0.003197,0.000000,0.045248,0.192821,0.00000,0.000000


In [12]:
def get_X_cell():
    return cell_GEP_mapping.drop('GEP 15 (DNA Replication)', axis=1)

def get_y_cell():
    return cell_GEP_mapping['GEP 15 (DNA Replication)']

In [13]:
X_cell_train, X_cell_test, y_cell_train, y_cell_test = train_test_split(get_X_cell(), get_y_cell(), test_size=0.2)

# alphas = np.logspace(-6, 6, 500)
# l1_ratios = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.97, 1.0]

# cell_elastic = ElasticNetCV(alphas=alphas, l1_ratio=l1_ratios, cv=10, n_jobs=-1)
# cell_elastic.fit(X_cell_train, y_cell_train)

In [14]:
# print(f'optimal cell alpha: {cell_elastic.alpha_}')
# print(f'optimal cell l1_ratio: {cell_elastic.l1_ratio_}')
# print(f'elastic net cell test R^2: {cell_elastic.score(X_cell_test, y_cell_test)}')

In [15]:
cell_LR = LinearRegression()

cell_LR.fit(X_cell_train, y_cell_train)

LinearRegression()

In [16]:
print(f'base linear regression cell test R^2: {cell_LR.score(X_cell_test, y_cell_test)}')

base linear regression cell test R^2: 1.0


In [17]:
print(f'MSE for cell model: {mean_squared_error(y_cell_test, cell_LR.predict(X_cell_test))}')

MSE for cell model: 8.280861883313566e-19


In [18]:
cell_weights = pd.DataFrame({
    'Feature': X_cell_train.columns,
    'Weight': cell_LR.coef_
})
cell_weights = pd.concat(
    [cell_weights, pd.DataFrame({'Feature': ['Intercept'], 'Weight': [cell_LR.intercept_]})]
)
cell_weights

,Feature,Weight
0,GEP 1 (Mono/DC),-1.0
1,GEP 2 (single donor-specific),-1.0
2,GEP 3,-1.0
3,GEP 4 (Elderly-HSC),-1.0
4,GEP 5,-1.0
5,GEP 6,-1.0
6,GEP 7,-1.0
7,GEP 8,-1.0
8,GEP 9,-1.0
9,GEP 10,-1.0
